## data preprocess

##### for augmented data:

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# # Define your data directory
# data_dir = 'Swimcat'
# # Create an ImageDataGenerator
# datagen = ImageDataGenerator(
#     rescale=1./255, validation_split=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     rotation_range=20,  # You can adjust the rotation range
#     brightness_range=[0.8, 1.2]
# )

# # Load the training dataset
# train_generator = datagen.flow_from_directory(
#     data_dir,
#     target_size=(224, 224),  # Adjust the target size according to your model requirements
#     batch_size=32,
#     class_mode='categorical',  # 'binary' for binary classification
#     subset='training'
# )

# # Load the validation dataset
# validation_generator = datagen.flow_from_directory(
#     data_dir,
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical',
#     subset='validation'
# )

# datagen_1 = ImageDataGenerator(rescale=1./255)
    
# test_generator = datagen_1.flow_from_directory(
#     'Swimcat_test',
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical',

    
# )

##### for non-augmented data:

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define your data directory
data_dir = 'Swimcat'
# Create an ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255, validation_split=0.2,

)

# Load the training dataset
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # Adjust the target size according to your model requirements
    batch_size=32,
    class_mode='categorical',  # 'binary' for binary classification
    subset='training'
)

# Load the validation dataset
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

datagen_1 = ImageDataGenerator(rescale=1./255)
test_generator = datagen_1.flow_from_directory(
    'Swimcat_test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    
)


Found 1536 images belonging to 6 classes.
Found 378 images belonging to 6 classes.
Found 186 images belonging to 6 classes.


In [2]:
import numpy as np
def load(data):
    train_x=[]
    train_y=[]
    for i in range(len(data)):
        image,label=data[i]
        train_x.extend(image)
        train_y.extend(label)
    return np.array(train_x),np.array(train_y)



In [3]:
train_x,train_y=load(train_generator)
val_x,val_y=load(validation_generator)
test_x,test_y=load(test_generator)

In [4]:
def make_it_binary(prediction_arr):
    for arr in prediction_arr:
        max_index = np.argmax(arr)
        arr[:] = 0
        arr[max_index] = 1
    return prediction_arr

In [5]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
def evaluate(test_p,actual,name='model',all=True):
    test=make_it_binary(test_p)
    print(f'{name} accuracy: {accuracy_score(actual,test)}\n')
    if all==True:
        for i in range(6):
            print(f'{name} accuracy for class {i+1}: {accuracy_score(actual[:,i],test[:,i])}')
            print(f'{name} precision for class {i+1}: {precision_score(actual[:,i],test[:,i])}')
            print(f'{name} recall for class {i+1}: {recall_score(actual[:,i],test[:,i])}')
            print(f'{name} f1 score for class {i+1}: {f1_score(actual[:,i],test[:,i])}\n')
    

## models

#### vgg16

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
# from tensorflow.keras.applications import AlexNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load pre-trained AlexNet model without top (final fully connected layers)
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for your specific task
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')  # num_classes is the number of classes in your task
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

model.fit(train_x,train_y,epochs=20)


In [ ]:
import tensorflow as tf
# model.save('vgg16_model.h5')
model = tf.keras.models.load_model("vgg16_model.h5")



In [ ]:
# prediction=model.predict(train_x)
# val_prediction=model.predict(val_x)
# test_prediction=model.predict(test_x)
evaluate(val_prediction,val_y,'val',all=False)
# evaluate(prediction,train_y,'train',all=False)
evaluate(test_prediction,test_y,'test',all=False)

In [ ]:
from sklearn.metrics import accuracy_score
print(f'training accuracy: {accuracy_score(train_y,prediction)}')
print(f'validation accuracy: {accuracy_score(val_y,val_prediction)}')

# training accuracy: 0.998046875
# validation accuracy: 0.9312169312169312
# test accuracy: 0.956989247311828

## resnet 50

In [ ]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.resnet50V2 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout, MaxPooling2D,Conv2D, Activation, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# Load the pre-trained ResNet50 model without the top (fully connected) layers
base_model = ResNet50V2(weights='imagenet', include_top=False,input_shape=(224,224,3))

# Add a custom top (fully connected) layer for your specific task
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x=  Dropout(0.5)(x)
predictions = Dense(6, activation='softmax')(x)  # Adjust 'num_classes' based on your task

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

# Freeze the weights of the pre-trained layers

# Compile the model with an appropriate optimizer, loss, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()


In [ ]:
model.fit(train_x, train_y, epochs=20)

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model("resnet50_v2_model.h5")
# prediction=model.predict(train_x)
# val_prediction=model.predict(val_x)
test_prediction=model.predict(test_x)
evaluate(test_prediction,test_y,'test',False)

In [ ]:
# for non augmented
# training accuracy: 1.0
# validation accuracy: 0.9074074074074074
# test accuracy: 0.8978494623655914


# for augmented data:
# test accuracy: 0.9301075268817204

# test accuracy for class 1: 0.9946236559139785
# test precision for class 1: 1.0
# test recall for class 1: 0.967741935483871
# test f1 score for class 1: 0.9836065573770492

# test accuracy for class 2: 0.9838709677419355
# test precision for class 2: 0.9117647058823529
# test recall for class 2: 1.0
# test f1 score for class 2: 0.9538461538461539

# test accuracy for class 3: 0.9623655913978495
# test precision for class 3: 0.8529411764705882
# test recall for class 3: 0.9354838709677419
# test f1 score for class 3: 0.8923076923076922

# test accuracy for class 4: 0.978494623655914
# test precision for class 4: 0.9655172413793104
# test recall for class 4: 0.9032258064516129
# test f1 score for class 4: 0.9333333333333333

# test accuracy for class 5: 0.9623655913978495
# test precision for class 5: 1.0
# test recall for class 5: 0.7741935483870968
# test f1 score for class 5: 0.8727272727272727

# test accuracy for class 6: 0.978494623655914
# test precision for class 6: 0.8857142857142857
# test recall for class 6: 1.0
# test f1 score for class 6: 0.9393939393939393




## inception model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
# from tensorflow.keras.applications import AlexNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load pre-trained AlexNet model without top (final fully connected layers)
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for your specific task
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')  # num_classes is the number of classes in your task
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

model.fit(train_x,train_y,epochs=20)

In [ ]:
import tensorflow as tf
# model.save('inception_v3_model.h5')
# model = tf.keras.models.load_model("inception_v3_model.h5")
# prediction=model.predict(train_x)
# val_prediction=model.predict(val_x)
test_prediction=model.predict(test_x)

# evaluate(val_prediction,val_y,'val',all=False)
# evaluate(prediction,train_y,'train',all=False)
evaluate(test_prediction,test_y,'test',all=False)


In [ ]:

# for non augmented
#train_accu= 1
# val_accu=0.88
# test accuracy: 0.8924731182795699

# for augmented:
# val accuracy: 0.9047619047619048
# train accuracy: 0.9986979166666666
# test accuracy: 0.8978494623655914

### mobilenet v2



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 with pre-trained weights (excluding the top classification layer)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top model for fine-tuning
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(6, activation='softmax')(x)

# Combine the base model with the custom top model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()



In [ ]:
model.fit(train_x,train_y,epochs=20,validation_data=[val_x,val_y])

In [8]:
import tensorflow as tf
model = tf.keras.models.load_model("mobilenet_v2_model_after_aug.h5")
# prediction=model.predict(train_x)
val_prediction=model.predict(val_x)
test_prediction=model.predict(test_x)

# evaluate(prediction,train_y,'train',all=False)
evaluate(val_prediction,val_y,'val',all=False)
evaluate(test_prediction,test_y,'test',all=False)


# model.save('mobilenet_v2_model_after_aug.h5')

# for non augmented:
# training accuracy: 1.0
# validation accuracy: 0.9206349206349206

# for augmented:
# train accuracy: 0.9986979166666666
# val accuracy: 0.9338624338624338
# test accuracy: 0.9247311827956989

6/6 [==============================] - 2s 306ms/step
val accuracy: 0.917989417989418

test accuracy: 0.9247311827956989



## densenet

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load DenseNet121 with pre-trained weights (excluding the top classification layer)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top model for fine-tuning
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(6, activation='softmax')(x)

# Combine the base model with the custom top model
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# Display the model summary
model.summary()



In [ ]:

model.fit(train_x,train_y,epochs=25,validation_data=[val_x,val_y])

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('dense_net_model.h5')


prediction=model.predict(train_x)
val_prediction=model.predict(val_x)
test_prediction=model.predict(test_x)


evaluate(prediction,train_y,'train')
# model.save('dense_net_model_after_aug.h5')

# before augmentation:
# training accuracy: 0.9993489583333334
# validation accuracy: 0.9470899470899471
# test accuracy: 0.9301075268817204


# test accuracy for class 1: 0.989247311827957
# test precision for class 1: 1.0
# test recall for class 1: 0.9354838709677419
# test f1 score for class 1: 0.9666666666666666

# test accuracy for class 2: 0.9946236559139785
# test precision for class 2: 0.96875
# test recall for class 2: 1.0
# test f1 score for class 2: 0.9841269841269841

# test accuracy for class 3: 0.967741935483871
# test precision for class 3: 0.8571428571428571
# test recall for class 3: 0.967741935483871
# test f1 score for class 3: 0.909090909090909

# test accuracy for class 4: 0.967741935483871
# test precision for class 4: 0.9310344827586207
# test recall for class 4: 0.8709677419354839
# test f1 score for class 4: 0.9

# test accuracy for class 5: 0.967741935483871
# test precision for class 5: 0.9629629629629629
# test recall for class 5: 0.8387096774193549
# test f1 score for class 5: 0.896551724137931

# test accuracy for class 6: 0.9731182795698925
# test precision for class 6: 0.8823529411764706
# test recall for class 6: 0.967741935483871
# test f1 score for class 6: 0.923076923076923


# after augmentatioN:
# training accuracy: 0.9934895833333334
# validation accuracy: 0.9603174603174603
# # test accuracy: 0.9001075268817204


## nasnet

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load NasNetMobile pre-trained on ImageNet data
base_model = NASNetMobile(weights='imagenet', include_top=False,input_shape=(224, 224, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add fully connected layers
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)

# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.fit(train_x,train_y,epochs=20)

In [ ]:
prediction=model.predict(train_x)
val_prediction=model.predict(val_x)
test_prediction=model.predict(test_x)

evaluate(val_prediction,val_y,'val',all=False)
evaluate(prediction,train_y,'train',all=False)
evaluate(test_prediction,test_y,'test',all=False)


model.save('nas_net_model_after_aug.h5')


# for non augmented:
# training accuracy: 1.0
# validation accuracy: 0.9338624338624338


# for augmented:
# val accuracy: 0.9021164021164021
# train accuracy: 1.0
# test accuracy: 0.8924731182795699

In [ ]:
# non augmented
# vgg16
# training accuracy: 0.998046875
# validation accuracy: 0.9312169312169312

# resnet50
# training accuracy: 1.0
# validation accuracy: 0.9074074074074074

#inception model
#train_accu= 1
# val_accu=0.88

# mobilenet_v2:
# training accuracy: 1.0
# validation accuracy: 0.9206349206349206

# dense net
# training accuracy: 0.9993489583333334
# validation accuracy: 0.9470899470899471

#nasnet
# training accuracy: 1.0
# validation accuracy: 0.9338624338624338


#augmented

# Dense Net:
# training accuracy: 0.9934895833333334
# validation accuracy: 0.9603174603174603

In [ ]:
# data augmentaion,data extend
# dropout layer increase

# augmented